The king needs our help. He has participated in a RC car competetion, and he has 5 cars to choose from. This is no ordinary RC car race it is a probabilistic one where each car move a random distance based on a set normal distribution(with standard deviation 1 and different means ) at each time step. The king has 5 cars to choose from but he has no idea what distribution do those car hold. He obviously cant test all cars at once(he can only control one car at a time), as he dosent trust anyone with his cars he asks us to find a way to test them and find the best car. 

We can solve this using K-armed bandit problem where at each time step the distance travelled by the car is the reward received and our job is to maximize reward at each time step. 

In [2]:
import pybullet as p
import time
import numpy as np
import pybullet_data 
import matplotlib.pyplot as plt
from collections import Counter
import random

pybullet build time: Dec 17 2023 23:51:54


For each of the car we need a distribution by which the car moves

We will be employing Optimistic initial values and sample avereging technique 

We will be making a 2 distribution to make the problem challenging 

A random force will be put to based on another normal distribution which will be also a normal distribution.

In [2]:
# cars = 3
# means_velocity = np.random.normal(.1 , .1 , cars)
# means_velocity# All have standard deviarion 1

In [3]:
# cars = 5
# means_force = np.random.normal(0.5 , 1 , cars)
# means_force# All have standard deviarion 1

Each will run for around 1000 time steps after which the battery on the cars must be rechanrged. So we restard from time step 0. This cycle continues for over 2000 iterations

In [4]:
means_velocity = [0.12588366, 0.10463964, 0.15764252]

In [5]:
Q = [0.01 , 0.01 , 0.01 ]
N = [0 , 0 , 0 ]
epsilon = 0.01 
total_timesteps_to_run_simulation = 100
total_timesteps = 2000
iterations = 1000
picked_car = []
rewards_cars = { 0 : [] , 1: [ ]  , 2: []}

In [6]:
#Environment 
def move_car(racecars , picked_car_number , velocity , total_timesteps_to_run_simulation , time_to_stop=100):
    
    p.setJointMotorControlArray(racecars[picked_car_number], [2,3,5,7], p.VELOCITY_CONTROL,
                                    targetVelocities =  [velocity for i in range(4)] , forces = [1,1,1,1])

    for time_steps in range(total_timesteps_to_run_simulation):
        p.stepSimulation()
        if time_steps == total_timesteps_to_run_simulation-1:
            basev = p.getBaseVelocity(racecars[picked_car_number])
            reward = basev[0][0]#the reward is the cars base velocity which can be observed by the agent

    #Stop the car from moving

    p.setJointMotorControlArray(racecars[picked_car_number], [2,3,5,7], p.VELOCITY_CONTROL,
                                    targetVelocities =  [0 for i in range(4)] , forces = [1,1,1,1])

    for time_steps in range(time_to_stop):
        p.stepSimulation()

    return reward
    
def get_velocity(picked_car):
    # we need to pick the reward for a car ; 
    #we need two means one for the velocity of the joints of the car and one is a random force from a car. 
    '''
    Parameters
    ---------------------
    picked_car - The index of the picked car

    Returns 
    ---------------------
    velocity - velocity of the joints of the car
    force - external force added to the cars base.
    '''

    applied_velocity = np.random.normal(means_velocity[picked_car] , 1 , 1)
    
    return applied_velocity[0] 

def pick_car(Q,epsilon):#epsilon is the probablity here
    pick = random.choices(['pick_max_car','pick_random'], weights=(1-epsilon,epsilon))
    if pick[0]=='pick_max_car':
        return Q.index(max(Q))
    else:
        max_car = Q.index(max(Q))
        a = [0,1,2]
        a.pop(max_car)
        return random.choices(a, weights=(0.5,0.5))[0]
    

In [7]:
# def give_reward

In [ ]:


epislon = 0.01 

# random_pick = total_timesteps * epislon

physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
#or p.DIRECT for non-graphical version p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally p.setGravity(0,0,-10)

for steps in range(iterations):
    print(steps,"/1000 is going on")
    planeId = p.loadURDF("plane.urdf")
    p.setGravity(0,0,-10)
    startPos1 = [0,1,0]
    startOrientation1 = p.getQuaternionFromEuler([0,0,0])
    racecar0 = p.loadURDF("/racecar/racecar.urdf",startPos1, startOrientation1) #set the center of mass frame (loadURDF sets base link frame) 
    p.resetBasePositionAndOrientation(racecar0, startPos1, startOrientation1)
    
    #car2
    startPos2 = [0,0,0]
    startOrientation2 = p.getQuaternionFromEuler([0,0,0])
    racecar1 = p.loadURDF("/racecar/racecar.urdf",startPos2, startOrientation2) #set the center of mass frame (loadURDF sets base link frame) 
    p.resetBasePositionAndOrientation(racecar1, startPos2, startOrientation2)
    
    
    startPos3 = [0,-1,0]
    startOrientation3 = p.getQuaternionFromEuler([0,0,0])
    racecar2 = p.loadURDF("/racecar/racecar.urdf",startPos3, startOrientation3) #set the center of mass frame (loadURDF sets base link frame) 
    p.resetBasePositionAndOrientation(racecar2, startPos3, startOrientation3)
    
    
    
    # startPos4 = [0,-1,0]
    # startOrientation4 = p.getQuaternionFromEuler([0,0,0])
    # racecar3 = p.loadURDF("/racecar/racecar.urdf",startPos4, startOrientation4) #set the center of mass frame (loadURDF sets base link frame) 
    # p.resetBasePositionAndOrientation(racecar3, startPos4, startOrientation4)
    
    
    # startPos5 = [0,-2,0]
    # startOrientation5 = p.getQuaternionFromEuler([0,0,0])
    # racecar4 = p.loadURDF("/racecar/racecar.urdf",startPos5, startOrientation5) #set the center of mass frame (loadURDF sets base link frame) 
    # p.resetBasePositionAndOrientation(racecar4, startPos5, startOrientation5)
    
   
    
    p.setRealTimeSimulation(0)
    
    racecars = [racecar0 , racecar1 , racecar2 ]
    
    for car in racecars:
        numJoints = p.getNumJoints(car)
        
        for joint in range(numJoints):
          p.changeDynamics(car,joint,linearDamping=0,angularDamping=0)
    
    for i in range(total_timesteps):
        
            
        car_number = pick_car(Q,epsilon)
        velocity = get_velocity(car_number)
        N[car_number]+=1

        picked_car.append(car_number)
        
        reward = move_car(racecars , car_number , velocity , total_timesteps_to_run_simulation)
        rewards_cars[car_number].append(reward)
        Q[car_number] = Q[car_number] + ((reward - Q[car_number])/N[car_number])

        print(i , ": iteration is going on")

    
    Q_numpy = np.array(Q)
    np.save('./save2/{}'.format(steps),Q_numpy)
    print(N)
    N_numpy = np.array(N)
    np.save('./save2/{}_N'.format(steps),N_numpy)
    
    print(Q)
    
    # for step in range(4000):
    #     p.stepSimulation()
    
    
    # # for i in range (10000):
        
        
    # # cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId) 
    # print(cubePos,cubeOrn)
    
    p.resetSimulation()
p.disconnect()

0 /1000 is going on
0 : iteration is going on
1 : iteration is going on
2 : iteration is going on
3 : iteration is going on
4 : iteration is going on
5 : iteration is going on
6 : iteration is going on
7 : iteration is going on
8 : iteration is going on
9 : iteration is going on
10 : iteration is going on
11 : iteration is going on
12 : iteration is going on
13 : iteration is going on
14 : iteration is going on
15 : iteration is going on
16 : iteration is going on
17 : iteration is going on
18 : iteration is going on
19 : iteration is going on
20 : iteration is going on
21 : iteration is going on
22 : iteration is going on
23 : iteration is going on
24 : iteration is going on
25 : iteration is going on
26 : iteration is going on
27 : iteration is going on
28 : iteration is going on
29 : iteration is going on
30 : iteration is going on
31 : iteration is going on
32 : iteration is going on
33 : iteration is going on
34 : iteration is going on
35 : iteration is going on
36 : iteration is 

In [5]:
N = np.load('./save2/355_N.npy')

In [6]:
N

array([  7506,   3553, 700941])